In [2]:
import gensim
from gensim import corpora, models
from sklearn.externals import joblib
from datetime import datetime
from gensim.models import CoherenceModel, LsiModel, HdpModel
import pickle
import glob
import matplotlib.pyplot as plt


from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet

import pyLDAvis
import pyLDAvis.gensim

import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
%%time 

# dataset = 'karbala'
dataset = 'all'

corpus_dictionary_file = 'data/eos/ngram/bigram_dict_'+dataset+'.dict'
corpus_doc2bow_file = 'data/eos/ngram/bigram_bow_corpus_'+dataset+'.mm'


# Load to memory
corpus = gensim.corpora.MmCorpus(corpus_doc2bow_file)
dictionary = gensim.corpora.Dictionary.load(corpus_dictionary_file)

print(corpus)
print(dictionary)

2017-06-21 17:43:17,259 : INFO : loaded corpus index from data/eos/ngram/bigram_bow_corpus_all.mm.index
2017-06-21 17:43:17,260 : INFO : initializing corpus reader from data/eos/ngram/bigram_bow_corpus_all.mm
2017-06-21 17:43:17,261 : INFO : accepted corpus with 459977 documents, 100000 features, 86838816 non-zero entries
2017-06-21 17:43:17,261 : INFO : loading Dictionary object from data/eos/ngram/bigram_dict_all.dict
2017-06-21 17:43:17,299 : INFO : loaded data/eos/ngram/bigram_dict_all.dict


MmCorpus(459977 documents, 100000 features, 86838816 non-zero entries)
Dictionary(100000 unique tokens: ['sco', 'قابل_توجهی', 'dog_accessibility', 'auschwitz_birkenau', 'judicial_inquiry']...)
CPU times: user 64 ms, sys: 20 ms, total: 84 ms
Wall time: 80.1 ms


Topic modeling with LSI

In [4]:
%%time


def lsi_model_build(lsi_model_path):
    print ("start modeling lsi " + datetime.now().strftime("%H:%M:%S"))
    lsimodel = LsiModel(corpus=corpus, num_topics=10, id2word=dictionary)
    joblib.dump(lsimodel, lsi_model_path)
    lsimodel.show_topics(num_topics=10)  # Showing only the top 5 topics
    print ("finished modeling lsi " + datetime.now().strftime("%H:%M:%S"))
    return lsimodel

# lsimodel = joblib.load(lsi_model_path)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 4.29 µs


Topic modeling with HDP

In [5]:
%%time

def hdp_model_build(hdp_model_path):
    print ("start modeling hdp " + datetime.now().strftime("%H:%M:%S"))
    hdpmodel = HdpModel(corpus=corpus, id2word=dictionary)
    joblib.dump(hdpmodel, hdp_model_path)
    hdpmodel.show_topics()
    print ("finished modeling hdp " + datetime.now().strftime("%H:%M:%S"))
    return hdpmodel

# hdpmodel = joblib.load(hdp_model_path)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.96 µs


In [6]:
%%time


def mallet_lda_build(mallet_model_path):
    mallet_bin = '/home/sonic/sonic/mallet-2.0.8/bin/mallet'
    

    lda_mallet_model = gensim.models.wrappers.LdaMallet(mallet_bin, corpus=corpus, 
                                               num_topics=20, id2word=dictionary)

    joblib.dump(lda_mallet_model, mallet_model_path)
    lda_mallet_model.show_topics()

# lda_mallet_model = joblib.load(mallet_model_path)


CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 4.05 µs


In [7]:
%%time 

lsi_model_path = 'data/eos/models/lsimodel_10_EOS.pkl'
hdp_model_path = 'data/eos/models/hdpmodel_EOS.pkl'
mallet_model_path = 'data/eos/models/malletmodel_EOS.pkl'

def generateModels():
    
    lsi_model_build(lsi_model_path)
    hdp_model_build(hdp_model_path)
    mallet_lda_build(mallet_model_path)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 3.81 µs


In [8]:
%%time

# Generate and save models
generateModels()


2017-06-21 17:44:05,302 : INFO : using serial LSI version on this node
2017-06-21 17:44:05,303 : INFO : updating model with new documents


start modeling lsi 17:44:05


2017-06-21 17:44:11,890 : INFO : preparing a new chunk of documents
2017-06-21 17:44:12,376 : INFO : using 100 extra samples and 2 power iterations
2017-06-21 17:44:12,376 : INFO : 1st phase: constructing (100000, 110) action matrix
2017-06-21 17:44:12,856 : INFO : orthonormalizing (100000, 110) action matrix
2017-06-21 17:44:19,354 : INFO : 2nd phase: running dense svd on (110, 20000) matrix
2017-06-21 17:44:19,776 : INFO : computing the final decomposition
2017-06-21 17:44:19,779 : INFO : keeping 10 factors (discarding 37.412% of energy spectrum)
2017-06-21 17:44:19,818 : INFO : processed documents up to #20000
2017-06-21 17:44:19,847 : INFO : topic #0(1675.304): 0.418*"turkish" + 0.397*"istanbul" + 0.361*"turkey" + 0.137*"police" + 0.121*"2014" + 0.116*"protest" + 0.112*"city" + 0.109*"continue" + 0.102*"border" + 0.098*"people"
2017-06-21 17:44:19,850 : INFO : topic #1(1037.634): 0.245*"city" + -0.237*"turkish" + -0.225*"istanbul" + -0.200*"turkey" + 0.162*"one" + 0.142*"2014" + 0.

2017-06-21 17:45:19,394 : INFO : topic #1(2134.282): -0.191*"turkish" + -0.184*"istanbul" + 0.172*"one" + 0.158*"city" + -0.156*"turkey" + 0.149*"2014" + 0.144*"also" + 0.135*"can" + 0.132*"people" + 0.128*"make"
2017-06-21 17:45:19,398 : INFO : topic #2(1104.295): 0.309*"new_york" + 0.274*"police" + 0.263*"protest" + 0.248*"grand_jury" + 0.199*"decision" + 0.189*"2014" + 0.182*"garner" + 0.165*"police_officer" + 0.165*"city" + 0.159*"death"
2017-06-21 17:45:19,401 : INFO : topic #3(1026.629): 0.245*"game" + -0.210*"report" + -0.193*"cia" + -0.191*"government" + 0.168*"win" + 0.158*"1" + 0.156*"3" + 0.148*"city" + 0.144*"point" + 0.133*"2"
2017-06-21 17:45:19,404 : INFO : topic #4(853.442): 0.407*"cia" + 0.345*"report" + -0.165*"city" + 0.162*"1" + 0.153*"game" + -0.147*"people" + 0.144*"2" + 0.140*"point" + 0.131*"3" + 0.130*"u.s."
2017-06-21 17:45:25,818 : INFO : preparing a new chunk of documents
2017-06-21 17:45:26,287 : INFO : using 100 extra samples and 2 power iterations
2017-06

2017-06-21 17:46:18,937 : INFO : topic #4(1061.326): 0.473*"cia" + 0.384*"report" + 0.175*"game" + 0.150*"point" + 0.141*"detainee" + -0.140*"city" + -0.138*"people" + 0.132*"torture" + -0.124*"government" + 0.113*"release"
2017-06-21 17:46:25,802 : INFO : preparing a new chunk of documents
2017-06-21 17:46:26,297 : INFO : using 100 extra samples and 2 power iterations
2017-06-21 17:46:26,298 : INFO : 1st phase: constructing (100000, 110) action matrix
2017-06-21 17:46:26,781 : INFO : orthonormalizing (100000, 110) action matrix
2017-06-21 17:46:33,288 : INFO : 2nd phase: running dense svd on (110, 20000) matrix
2017-06-21 17:46:33,693 : INFO : computing the final decomposition
2017-06-21 17:46:33,696 : INFO : keeping 10 factors (discarding 42.221% of energy spectrum)
2017-06-21 17:46:33,732 : INFO : merging projections: (100000, 10) + (100000, 10)
2017-06-21 17:46:33,758 : INFO : keeping 10 factors (discarding 1.744% of energy spectrum)
2017-06-21 17:46:33,772 : INFO : processed docum

...

2017-06-21 19:40:50,846 : INFO : (7, '0.010*win + 0.009*city + 0.009*1 + 0.009*goal + 0.008*game + 0.007*team + 0.006*first + 0.006*season + 0.006*chelsea + 0.005*0')
2017-06-21 19:40:50,928 : INFO : (8, '0.049*die + 0.034*der + 0.029*und + 0.013*al + 0.012*von + 0.010*das + 0.009*ist + 0.009*auch + 0.008*war + 0.008*sich')
2017-06-21 19:40:51,009 : INFO : (9, '0.012*uae + 0.009*2014 + 0.008*al + 0.007*december + 0.007*country + 0.006*minister + 0.006*national + 0.006*president + 0.006*full_story + 0.006*read')
2017-06-21 19:40:51,089 : INFO : (10, '0.035*şi + 0.022*care + 0.015*la + 0.009*în + 0.008*s + 0.008*al + 0.007*un + 0.005*cu + 0.005*nu + 0.005*articole_de')
2017-06-21 19:40:51,170 : INFO : (11, '0.018*et + 0.016*la + 0.009*le + 0.008*al + 0.007*que + 0.007*des + 0.007*e + 0.007*un + 0.006*2014 + 0.006*à')
2017-06-21 19:40:51,250 : INFO : (12, '0.012*این + 0.007*در_این + 0.007*برای + 0.006*است_که + 0.005*است. + 0.005*که_در + 0.004*است_و + 0.004*را_به + 0.004*نیز + 0.003*را_در'

2017-06-21 19:42:31,233 : INFO : (10, '0.035*şi + 0.022*care + 0.015*la + 0.009*în + 0.008*s + 0.008*al + 0.007*un + 0.005*cu + 0.005*nu + 0.005*articole_de')
2017-06-21 19:42:31,316 : INFO : (11, '0.018*et + 0.016*la + 0.008*le + 0.008*al + 0.007*que + 0.007*des + 0.007*e + 0.007*un + 0.006*2014 + 0.006*y')
2017-06-21 19:42:31,399 : INFO : (12, '0.012*این + 0.007*در_این + 0.007*برای + 0.006*است_که + 0.005*است. + 0.005*که_در + 0.004*است_و + 0.004*را_به + 0.004*نیز + 0.003*را_در')
2017-06-21 19:42:31,482 : INFO : (13, '0.022*new_york + 0.017*garner + 0.016*grand_jury + 0.015*police + 0.015*protest + 0.015*city + 0.013*decision + 0.013*police_officer + 0.011*death + 0.011*man')
2017-06-21 19:42:31,565 : INFO : (14, '0.010*2014 + 0.006*december + 0.006*bank + 0.006*late + 0.006*show + 0.006*read + 0.006*dan + 0.005*wednesday + 0.004*tuesday + 0.004*full_story')
2017-06-21 19:42:31,647 : INFO : (15, '0.013*إلى + 0.013*أن + 0.008*أو + 0.004*إن + 0.004*إنشاء_تنبيه + 0.003*لبنان + 0.003*داعش 

finished modeling hdp 19:42:35


2017-06-21 19:46:57,976 : INFO : converting temporary corpus to MALLET format with /home/sonic/sonic/mallet-2.0.8/bin/mallet import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input /tmp/795bc3_corpus.txt --output /tmp/795bc3_corpus.mallet
2017-06-21 19:47:54,877 : INFO : training MALLET LDA with /home/sonic/sonic/mallet-2.0.8/bin/mallet train-topics --input /tmp/795bc3_corpus.mallet --num-topics 20  --alpha 50 --optimize-interval 0 --num-threads 4 --output-state /tmp/795bc3_state.mallet.gz --output-doc-topics /tmp/795bc3_doctopics.txt --output-topic-keys /tmp/795bc3_topickeys.txt --num-iterations 1000 --inferencer-filename /tmp/795bc3_inferencer.mallet --doc-topics-threshold 0.0
2017-06-21 21:16:54,145 : INFO : loading assigned topics from /tmp/795bc3_state.mallet.gz


CPU times: user 3h 8min 42s, sys: 5h 14min 24s, total: 8h 23min 6s
Wall time: 3h 43min 11s


In [9]:
## Experiments

from multiprocessing import Pool


docs_filepath = 'data/eos/ngram/bigram_transformed_reviews_'+dataset+'.txt'


corpus_text = []
with open(docs_filepath) as f:
    for line in f:
        corpus_text.append(line.split(u' '))
    
def load_topic_models(model_list):
    
    model_list = []
    # Load the models    
    for fname in model_list:
        model = joblib.load(fname)
        model_list.append(model)

    return model_list
    
    
def eval_topic_coherence(topic_model_path):
    
    model = joblib.load(topic_model_path)
    print ("Starting coherence calculation " + datetime.now().strftime("%H:%M:%S"))
    print (model)
    # Get model topics     
    model_topics = model.show_topics(formatted=False)
    # Generate topic list     
    topics = [[word for word, prob in topic] for topicid, topic in model_topics]
    print(topics)
    coherence = CoherenceModel(topics=topics, corpus=corpus, texts=corpus_text, 
                               dictionary=dictionary, window_size=10, coherence='u_mass').get_coherence()
    
    print ("finished model coherence {}".format(coherence) + datetime.now().strftime("%H:%M:%S"))
   
    return [topic_model_path, coherence]
    

def eval_execute():

            
    model_list = [lsi_model_path, hdp_model_path, mallet_model_path]
    print (model_list)
    # start 7 worker processes
    pool = Pool(processes=7)              
    coherence = pool.map(eval_topic_coherence, model_list)
    print(coherence)

    # Show graph
    indices = [x[0] for x in coherence]
    y= [x[1] for x in coherence]
    n = len(coherence)
    x = range(n)
    plt.bar(x, y, width=0.2, tick_label=indices, align='center')
    plt.xlabel('Models')
    plt.ylabel('Coherence Value')
    plt.show()
    

eval_execute()

['data/eos/models/lsimodel_10_EOS.pkl', 'data/eos/models/hdpmodel_EOS.pkl', 'data/eos/models/malletmodel_EOS.pkl']
Starting coherence calculation 22:47:26
[['israel', 'iran', 'egypt', 'israeli', 'official', 'yemen', 'al', 'saudi_arabia', 'security', 'saudi'], ['game', 'canada', '’', 'time', 'make', 'play', 'canadian', 'team', 'season', 'good'], ['health', 'study', 'hospital', 'research', 'patient', 'people', 'find', 'report', 'child', 'care'], ['turkey', '2014', 'turkish', 'dec', '2015', 'visit', 'child', 'official', 'world', 'president'], ['market', 'price', 'million', 'oil', 'company', 'percent', 'bank', 'low', 'share', 'billion'], ['report', 'cia', 'new_york', 'u.s.', 'obama', 'release', 'torture', 'official', 'decision', 'police_officer'], ['die', 'full_story', 'read', 'der', 'und', 'al', 'december', 'war', '2014', 'von'], ['city', 'people', 'area', 'home', 'road', 'water', 'building', 'resident', 'more_than', 'local'], ['iraq', 'syria', 'force', 'islamic_state', 'group', 'military

Process ForkPoolWorker-3:
Process ForkPoolWorker-2:
Process ForkPoolWorker-1:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Process ForkPoolWorker-4:
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
Process ForkPoolWorker-6:
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Process ForkPoolWorker-5:
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  F

KeyboardInterrupt: 

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 366, in _handle_workers
    pool._maintain_pool()
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 240, in _maintain_pool
    self._repopulate_pool()
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 233, in _repopulate_pool
    w.start()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 105, in start
    self._popen = self._Popen(self)
  File "/usr/lib/python3.5/multiprocessing/context.py", line 267, in _Popen
    return Popen(process_obj)
  File "/usr/lib/python3.5/multiprocessing/popen_fork.py", line 20, in __init__
    self._launch(process_obj)
  File "/usr/lib/python3.5/multiprocessing/popen_fork.py", line 67, in _launch
    self.pid = os.fork(

In [ ]:
%%time
coherence = [['lsimodel_10', 2.5318031449974874], 
 ['hdpmodel', 1.958798883672267], 
 ['mallet', 1.2456459799663451]]

# Show graph
indices = [x[0] for x in coherence]
y= [x[1] for x in coherence]
n = len(coherence)
x = range(n)
plt.bar(x, y, width=0.2, tick_label=indices, align='center')
plt.xlabel('Models')
plt.ylabel('Coherence Value')
plt.show()